Here we use Rnn Model to Find the Accurate Element and Perform Self Healing.

In [ ]:
!pip install lxml Beautifulsoup4

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Attention, TimeDistributed,Input

In [ ]:
df = pd.read_csv('html_examples_500.csv')
df.head()

,input,output
0,"<main>\n <p class=""introf-414"">This is intr...","<p class=""Artile-informations"" name=""article-n..."
1,"<main>\n <p class=""introf-153"">This is intr...","<p class=""Artile-informations"" name=""article-n..."
2,"<main>\n <p class=""introf-453"">This is intr...","<p class=""Artile-informations"" name=""article-n..."
3,"<main>\n <p class=""introf-398"">This is intr...","<p class=""Artile-informations"" name=""article-n..."
4,"<main>\n <p class=""introf-50"">This is intro...","<p class=""Artile-informations"" name=""article-n..."


In [ ]:
from lxml import etree

def parse_html_to_sentence(html):
    # Parse the HTML string
    parser = etree.HTMLParser()
    tree = etree.fromstring(html, parser)

    # Function to process each element
    def process_element(element):
        result = []
        # Append tag name
        result.append(element.tag)

        # Append attributes if any
        for attr, value in element.attrib.items():
            result.append(f'{attr}="{value}"')

        # Append text if any
        if element.text and element.text.strip():
            result.append(element.text.strip())

        return ' '.join(result)

    # Traverse the tree and collect the results
    results = []
    for elem in tree.iter():
        result = process_element(elem)
        if result:
            results.append(result)

    # Join all the results into a single sentence
    sentence = ' '.join(results)
    return sentence


# # Parse and convert to sentence
# sentence = parse_html_to_sentence(html)
# print(sentence)



In [ ]:
from lxml import etree

def parse_html_to_sentence2(html):
    # Parse the HTML string
    parser = etree.HTMLParser()
    tree = etree.fromstring(html, parser)

    # Function to process each element
    def process_element(element):
        result = []
        # Append tag name
        result.append(element.tag)

        # Append attributes if any
        for attr, value in element.attrib.items():
            result.append(f'{attr}="{value}"')

        # Append text if any
        if element.text and element.text.strip():
            result.append(element.text.strip())

        return ' '.join(result)

    # Traverse the tree and collect the results
    results = []
    for elem in tree.iter():
        result = process_element(elem)
        if result:
            results.append(result)

    # Join all the results into a single sentence
    sentence = ' '.join(results)
    return sentence.replace("html body","")


# # Parse and convert to sentence
# sentence = parse_html_to_sentence(html)
# print(sentence)



In [ ]:
df['ParsedInput'] = df['input'].apply(parse_html_to_sentence)
df['ParsedOutput'] = df['output'].apply(parse_html_to_sentence2)
df = df.drop(['input','output'], axis='columns')


In [ ]:
print(df.iloc[1,0])
print(df.iloc[1,1])

html body main p class="introf-153" This is introduction paragraph 153. section h2 Section Title 153 article h3 class="takeaway-153" Article Title 153 p class="Artile-informations" name="article-name-153" Article content 153.
 p class="Artile-informations" name="article-name-153" Article content 153.


In [ ]:
WholeHtml = df['ParsedInput'].tolist()
Req_element = df['ParsedOutput'].tolist()

In [ ]:
# Tokenize the paragraphs and sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(WholeHtml + Req_element)

In [ ]:
paragraph_sequences = tokenizer.texts_to_sequences(WholeHtml)
sentence_sequences = tokenizer.texts_to_sequences(Req_element)

In [ ]:
# Pad sequences
max_paragraph_length = max([len(seq) for seq in paragraph_sequences])
max_sentence_length = max([len(seq) for seq in sentence_sequences])
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
paragraph_sequences = pad_sequences(paragraph_sequences, maxlen=max_paragraph_length, padding='post')
sentence_sequences = pad_sequences(sentence_sequences, maxlen=max_sentence_length, padding='post')

In [ ]:

# Define model architecture
def build_model(vocab_size, max_paragraph_length, max_sentence_length):
    # Encoder
    encoder_inputs = Input(shape=(max_paragraph_length,))
    x = Embedding(input_dim=vocab_size, output_dim=128)(encoder_inputs)
    x = LSTM(128, return_sequences=False)(x)

    # Decoder
    decoder_inputs = Input(shape=(max_sentence_length,))
    y = Embedding(input_dim=vocab_size, output_dim=128)(decoder_inputs)
    y = LSTM(128, return_sequences=True)(y)
    decoder_outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(y)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
model = build_model(vocab_size, max_paragraph_length, max_sentence_length)

In [ ]:
# Prepare targets for training
decoder_input_data = np.zeros((sentence_sequences.shape[0], max_sentence_length))
decoder_output_data = np.expand_dims(sentence_sequences, -1)  # For sparse categorical crossentropy

In [ ]:
model.fit([paragraph_sequences, decoder_input_data], decoder_output_data, epochs=20, batch_size=2)

Epoch 1/20
500/500 [==============================] - 10s 15ms/step - loss: 2.0440 - accuracy: 0.6509
Epoch 2/20
500/500 [==============================] - 8s 16ms/step - loss: 1.1538 - accuracy: 0.8182
Epoch 3/20
500/500 [==============================] - 7s 14ms/step - loss: 1.1426 - accuracy: 0.8182
Epoch 4/20
500/500 [==============================] - 8s 17ms/step - loss: 1.1394 - accuracy: 0.8182
Epoch 5/20
500/500 [==============================] - 7s 14ms/step - loss: 1.1376 - accuracy: 0.8182
Epoch 6/20
500/500 [==============================] - 9s 18ms/step - loss: 1.1366 - accuracy: 0.8182
Epoch 7/20
500/500 [==============================] - 7s 14ms/step - loss: 1.1357 - accuracy: 0.8182
Epoch 8/20
500/500 [==============================] - 9s 17ms/step - loss: 1.1353 - accuracy: 0.8182
Epoch 9/20
500/500 [==============================] - 8s 15ms/step - loss: 1.1348 - accuracy: 0.8182
Epoch 10/20
500/500 [==============================] - 8s 15ms/step - loss: 1.1344 - accur

In [ ]:
def generate_sentence(paragraph):
    sequence = tokenizer.texts_to_sequences([paragraph])
    padded_sequence = pad_sequences(sequence, maxlen=max_paragraph_length, padding='post')

    # Create decoder input (initially all zeros)
    decoder_input_data = np.zeros((1, max_sentence_length))
    prediction = model.predict([padded_sequence, decoder_input_data])

    # Convert prediction to text
    predicted_sequence = np.argmax(prediction, axis=-1)
    predicted_sentence = tokenizer.sequences_to_texts(predicted_sequence)

    return predicted_sentence[0]


In [ ]:
# Example usage
paragraph = """html body main p class="introf-153" This is introduction paragraph 153. section h2 Section Title 153 article h3 class="takeaway-153" Article Title 153 p class="Artile-informations" name="article-name-153" Article content 153."""
print(generate_sentence(paragraph))

1/1 [==============================] - 1s 509ms/step
p class artile informations name article name 439 article content 439
